In [3]:
import sys
# adding llms to the system path, make sure to configure the file azureopenai in llms folder with end point details
sys.path.append('C:\\proddev\\research\\aiagentssamples\\llms\\')
import azureopenai

import autogen
from autogen import ConversableAgent

llm_config = {
    "seed": 42,  # for caching the response
    "config_list": azureopenai.config_list,
    "temperature": 0.5  # for coding, it is suggested to be less creative
}

writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": azureopenai.config_list},
    system_message="""
    You are a professional writer, known for your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    You should improve the quality of the content based on the feedback from the user.
    """,
)

user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

critic = autogen.AssistantAgent(
    name="Critic",
    llm_config={"config_list": azureopenai.config_list},
    system_message="""
    You are a critic, known for your thoroughness and commitment to standards.
    Your task is to scrutinize content for any harmful elements or regulatory violations, ensuring
    all materials align with required guidelines.
    For code
    """,
)


def reflection_message(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"Reflect and provide critique on the following writing. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


user_proxy.register_nested_chats(
    [{"recipient": critic, "message": reflection_message, "summary_method": "last_msg", "max_turns": 1}],
    trigger=writer,  # condition=my_condition,
)

task = """Write a concise but engaging blogpost about Navida."""


res = user_proxy.initiate_chat(recipient=writer, message=task, max_turns=2, summary_method="last_msg")

User (to Writer):

Write a concise but engaging blogpost about Navida.

--------------------------------------------------------------------------------
Writer (to User):

Certainly! Here’s an engaging and concise blog post about Navida:

---

### Unveiling Navida: A Glimpse Into the Future of Innovation

In a world where technology evolves at breakneck speed, Navida stands as a beacon of innovation, seamlessly blending cutting-edge advancements with user-centric design. But what makes Navida a name to watch? Let's embark on a brief journey to uncover the essence of this forward-thinking entity.

#### Pioneering Technological Excellence

At its core, Navida is dedicated to pushing the boundaries of what’s possible. With a diverse portfolio of solutions ranging from AI-driven applications to revolutionary hardware, Navida has positioned itself at the forefront of technological progress. Each product and service is a testament to their commitment to enhance the way we live, work, and int